In [4]:
#!/usr/bin/env python
# coding: utf-8

import rosbag, os, matplotlib, pickle
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
from qsrlib.qsrlib import QSRlib, QSRlib_Request_Message
from qsrlib_io.world_trace import Object_State, World_Trace
from qsrlib.qsrlib import QSR_QTC_BC_Simplified
import numpy as np
import pandas as pd
import datetime as dt

os.chdir("/home/loz/QTC_Trajectory_HMMs/from_bags/")

In [5]:
# In[2]:


lab_bags = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser("~/QTC_Trajectory_HMMs/from_bags/rt-test-bags/")) for f in fn]
lab_bags[0].split("/")[-1][:2]

'hf'

In [6]:
lab_bags = [lab_bag for lab_bag in lab_bags if lab_bag.split("/")[-1][:2] == "hf"]
lab_bags

['/home/loz/QTC_Trajectory_HMMs/from_bags/rt-test-bags/ul/7/hf_study_2020-08-07-17-08-49.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/rt-test-bags/ul/3/hf_study_2020-08-07-17-06-37.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/rt-test-bags/ul/10/hf_study_2020-08-07-17-10-21.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/rt-test-bags/ul/8/hf_study_2020-08-07-17-09-18.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/rt-test-bags/ul/4/hf_study_2020-08-07-17-07-03.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/rt-test-bags/ul/6/hf_study_2020-08-07-17-08-10.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/rt-test-bags/ul/9/hf_study_2020-08-07-17-09-46.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/rt-test-bags/ul/2/hf_study_2020-08-07-17-06-12.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/rt-test-bags/ul/1/hf_study_2020-08-07-17-04-47.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/rt-test-bags/ul/5/hf_study_2020-08-07-17-07-24.bag',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/rt-test

In [8]:
seqs = {"pbl": [], "pbr": [], "pcl": [], "pcr": [], "rotl": [], "rotr": [], "ul": [], "ur": []}

for bag_path in lab_bags:
    bag = rosbag.Bag(bag_path)
    
    seq = []
    sit_code = "_".join(bag_path.split("/")[-3:-2])
#     print(sit_code)
    for topic, msg, t in bag.read_messages(topics=['/robot5/qsr/qtc_state']):
        if len(seq) == 0:
            seq.append(msg.data)
        elif seq[-1] != msg.data:
            seq.append(msg.data)
    
#     print("Complete seq: " + str(seq))
    for state_ix in range(len(seq)):
#         print(seq[:state_ix+1])
        seqs[sit_code].append(seq[:state_ix+1])

seqs

{'pbl': [['-,-,-,-'],
  ['-,-,-,-'],
  ['-,-,-,-'],
  ['-,-,-,-', '+,+,-,-'],
  ['-,-,-,-'],
  ['-,-,-,-', '-,0,-,-'],
  ['-,-,-,-', '-,0,-,-', '+,+,-,-'],
  ['-,-,-,-'],
  ['-,-,-,-', '0,+,-,-'],
  ['-,-,-,-', '0,+,-,-', '+,+,-,-'],
  ['-,-,-,-'],
  ['-,-,-,-', '-,-,+,-'],
  ['-,-,-,-', '-,-,+,-', '-,-,-,-'],
  ['-,-,-,-', '-,-,+,-', '-,-,-,-', '+,+,-,-'],
  ['-,-,-,-'],
  ['-,-,-,-', '+,+,+,-'],
  ['-,-,-,-'],
  ['-,-,-,-', '-,0,-,-'],
  ['-,-,-,-', '-,0,-,-', '0,+,-,-'],
  ['-,-,-,-', '-,0,-,-', '0,+,-,-', '+,+,-,-'],
  ['-,-,-,-'],
  ['-,-,-,-', '+,+,-,-'],
  ['-,-,+,-'],
  ['-,-,+,-', '-,-,-,-']],
 'pbr': [['-,-,+,+'],
  ['-,-,+,+', '+,+,+,+'],
  ['-,-,+,+'],
  ['-,-,+,+', '-,0,+,0'],
  ['-,-,+,+', '-,0,+,0', '+,0,+,0'],
  ['-,-,+,+'],
  ['-,-,+,+', '-,+,+,+'],
  ['-,-,+,+'],
  ['-,-,+,+'],
  ['-,-,+,0'],
  ['-,-,+,0', '-,-,+,+'],
  ['-,-,+,0', '-,-,+,+', '-,0,+,+'],
  ['-,-,+,0', '-,-,+,+', '-,0,+,+', '-,+,+,+'],
  ['-,-,+,+'],
  ['-,-,+,+', '-,0,+,+'],
  ['-,-,+,+', '-,0,+,+', '

In [9]:
total_seqs = 0
for key in seqs.keys():
    total_seqs += len(seqs[key])
total_seqs

576

In [10]:
with open("hf_expert_qtc_seqs.pickle", "w") as f:
    pickle.dump(seqs, f)

In [11]:
with open("hf_expert_qtc_seqs.pickle", "r") as f:
    print(pickle.load(f))

{'pbr': [['-,-,+,+'], ['-,-,+,+', '+,+,+,+'], ['-,-,+,+'], ['-,-,+,+', '-,0,+,0'], ['-,-,+,+', '-,0,+,0', '+,0,+,0'], ['-,-,+,+'], ['-,-,+,+', '-,+,+,+'], ['-,-,+,+'], ['-,-,+,+'], ['-,-,+,0'], ['-,-,+,0', '-,-,+,+'], ['-,-,+,0', '-,-,+,+', '-,0,+,+'], ['-,-,+,0', '-,-,+,+', '-,0,+,+', '-,+,+,+'], ['-,-,+,+'], ['-,-,+,+', '-,0,+,+'], ['-,-,+,+', '-,0,+,+', '-,-,+,+'], ['-,-,+,+'], ['-,-,+,0'], ['-,-,+,0', '-,-,+,+'], ['-,-,+,0', '-,-,+,+', '-,0,+,+'], ['-,-,+,+']], 'pcr': [['-,-,+,-'], ['-,-,+,-', '+,-,+,0'], ['-,-,+,-', '+,-,+,0', '+,-,+,+'], ['-,-,+,-'], ['-,-,+,-', '0,-,+,-'], ['-,-,+,-', '0,-,+,-', '+,-,+,-'], ['-,-,+,-', '0,-,+,-', '+,-,+,-', '+,-,+,0'], ['-,-,+,-', '0,-,+,-', '+,-,+,-', '+,-,+,0', '+,-,+,+'], ['-,-,+,-'], ['-,-,+,-', '+,-,+,-'], ['-,-,+,-', '+,-,+,-', '+,-,+,0'], ['-,-,+,-'], ['-,-,+,-', '+,-,+,-'], ['-,-,+,-', '+,-,+,-', '+,-,+,0'], ['-,-,+,-'], ['-,-,+,-', '+,-,-,-'], ['-,-,+,-', '+,-,-,-', '-,-,+,-'], ['-,-,+,-', '+,-,-,-', '-,-,+,-', '+,-,+,-'], ['-,-,+,-', '